In [ ]:
__author__ = 'pratyush'
import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
all_params = cm_help.get_params()

In [ ]:
def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    # default = 
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    # print(json.dumps(environments, indent=2))
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    # print(jres['environments'][1]['environment_id'])
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    #print(colres['collections'])
    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

In [ ]:
discovery_config = discovery_init(COLLECTION_NAME='Naviga1',default="r3")

In [ ]:
offset = 0
count = 3000
reqparams = {'offset':offset,'count':count}
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)

In [ ]:
result = []
offset = 0

while offset<=261000:
    
    count = 1000
    reqparams = {'offset':offset,'count':count}
    discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)
    
    offset+=1000
    
    for i in range(1000):
        
        res={}
        
        doc = discovery_data["results"][i]
        res["id"]=doc["id"]
        
        source = discovery_data["results"][i]['html']
        batRegex = re.compile(r'([a-zA-Z0-9]{2}\s<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')
        
        lists = re.findall(batRegex,source)
        len_ = len(lists)
        for i in range(len_):
            lists[i] = list(lists[i])
        
        regex = re.compile(r'\svalue="(.*?)"')
        subst = ''
        for i in range(len_):
            bregex = re.findall(regex,lists[i][0])
            if(len(bregex)!=0):
                lists[i][0] = lists[i][0].replace(bregex[0], subst)

        regex = re.compile(r'\sidsrc="(.*?)"')
        for i in range(len_):
            bregex = re.findall(regex,lists[i][0])
            if(len(bregex)!=0):
                lists[i][0] = lists[i][0].replace(bregex[0], subst)
        
        for i in range(len_):
            lists[i][0]=lists[i][0].split('/')
            lists[i][0][1]=lists[i][0][1].split('>')
            keyword = lists[i][0][1][0]
            value = lists[i][1]
    
            if keyword in res:
                if value not in res[keyword]:
                    res[keyword].append(value)       
            else:
                res[keyword] = [value]
        
        result.append(res)

In [ ]:
p=json.dumps(result,indent=4)
print(p)